In [9]:
%%capture
import os
import desc.slrealizer
import pandas as pd
import numpy as np
from itertools import product
import galsim

In [10]:
realizer_path = os.path.join(os.path.expanduser('~'), 'stage', 'sl', 'SLRealizer')
twinkles_path = os.path.join(realizer_path, 'data', 'twinkles_observation_history.csv')
sdss_obj_path = os.path.join(realizer_path, 'data', 'sdss_object.csv')
sdss_src_path = os.path.join(realizer_path, 'data', 'source_catalog_sdss.csv')

In [11]:
NUM_LENSES = 15

In [12]:
# TODO merge constants with positive example code
attributes = ['x', 'y', 'flux', 'size', 'e1', 'e2'] # no psf_sigma or sky
bands = ['g', 'i', 'r', 'u', 'z']
keep_cols = ['lensID'] + sorted([b+'_'+a for a, b in product(attributes, bands)])

In [13]:
# TODO filter catalog to have similar properties as positive examples
catalog = pd.read_csv(filepath_or_buffer=sdss_obj_path)[keep_cols]\
            .head(NUM_LENSES)
observation = pd.read_csv(filepath_or_buffer=twinkles_path, index_col=0)\
                .query("(expMJD < 60919) & (filter != 'y')")\
                .as_matrix()

In [14]:
NUM_OBS = len(observation)

In [15]:
def draw_gal_image(currLens, currObs):
    MJD, band, PSF_HWHM, sky_mag = currObs[0], currObs[1], currObs[2], currObs[3]
    
    galaxy = galsim.Gaussian(half_light_radius=currLens[band+'_size'], 
                             flux=currLens[band+'_flux'])
    big_fft_params = galsim.GSParams(maximum_fft_size=10240)
    galaxy = galaxy.shift([currLens[band+'_x'], currLens[band+'_y']])
    galaxy = galaxy.shear(e1=currLens[band+'_e1'], e2=currLens[band+'_e2'])
    
    psf = galsim.Gaussian(flux=1.0, fwhm=2.*PSF_HWHM)
    obj = galsim.Convolve(galaxy, psf, gsparams=big_fft_params)
    img = obj.drawImage(scale=0.2)
    return img, obj

In [16]:
def create_row(currLens, currObs, manual_error=True):
    MJD, band, PSF_HWHM, sky_mag = currObs[0], currObs[1], currObs[2], currObs[3]
    
    img, obj = draw_gal_image(currLens, currObs)
    flux_err_calc = np.power(10, (22.5 - sky_mag)/2.5)/5. # because Fb = 5 \sigma_b
    try:
        shape_info = img.FindAdaptiveMom()
    except:
        print "HSM failed"
        return None
    nx, ny = img.array.shape
    # TODO make pixel scale globally configurable
    pixel_scale = 0.2 # arcsec/pixel
    # Calculate the real position from the arbitrary pixel position
    first_moment_x, first_moment_y = (shape_info.moments_centroid.x - 0.5*nx)*pixel_scale, \
                                     (shape_info.moments_centroid.y - 0.5*ny)*pixel_scale 
    # Total image intensity for best-fit elliptical Gaussian from adaptive moments
    flux = shape_info.moments_amp
    size = shape_info.moments_sigma * pixel_scale # unit of pixels is returned, so change units for arcsec
    lensID = currLens['lensID']
    
    e1 = shape_info.observed_shape.e1
    e2 = shape_info.observed_shape.e2
    
    if manual_error:
        size += size * noissify_data(desc.slrealizer.get_second_moment_err(), desc.slrealizer.get_second_moment_err_std())
        first_moment_x += noissify_data(desc.slrealizer.get_first_moment_err(), desc.slrealizer.get_first_moment_err_std()) * first_moment_x
        first_moment_y += noissify_data(desc.slrealizer.get_first_moment_err(), desc.slrealizer.get_first_moment_err_std()) * first_moment_y
        flux += flux*noissify_data(desc.slrealizer.get_flux_err(), desc.slrealizer.get_flux_err_std())
    # TODO consider just returning shear object, etc.
    params_dict = {'MJD': MJD, 'filter': band,
                   'x': first_moment_x,'y': first_moment_y, 
                   'flux': flux, 'flux_err': flux_err_calc, 'size': size, 'e1': e1, 'e2': e2,
                    'lensID': lensID}
    return params_dict

def noissify_data(mean, stdev):
    """
    Given a mean and a standard deviation of a measurement, add a noise to the data
    """
    return np.random.normal(loc=mean, scale=stdev)


In [17]:
keep_cols.append('MJD')
df = pd.DataFrame(columns=keep_cols)

print("number of systems: %d, number of observations: %d" %(NUM_LENSES, NUM_OBS))
# TODO get rid of nested for loop
hsm_failed = 0
for j in xrange(NUM_OBS):
    for i in xrange(NUM_LENSES):
        #if self.catalog.sample[i]['ELLIP'] < ellipticity_upper_limit: # ellipticity cut : 0.5
        params_dict = create_row(currLens=catalog.loc[i],\
                                 currObs=observation[j])
        if params_dict == None:
            hsm_failed += 1
        else:
            # FIXIT try not to use list comprehension...
            vals_arr = np.array([params_dict[k] for k in keep_cols])
            df.loc[len(df)]= vals_arr
    if j % 20 == 0:
        print("Done with %d observations..." %j)
df.set_index('lensid', inplace=True)
df.to_csv(save_dir, index=True)
print("Done making the source table, with %d errors from HSM failure." %hsm_failed)

number of systems: 15, number of observations: 210


ValueError: Requested distortion exceeds 1: 32.00020670428471

In [92]:
catalog.loc[0]

lensID     0.000000
g_e1      -0.115018
g_e2      32.000000
g_flux     7.626726
g_size     1.481102
g_x       -0.022303
g_y       -0.099109
i_e1      -0.150845
i_e2      32.000000
i_flux    28.261839
i_size     1.493424
i_x        0.028494
i_y       -0.047769
r_e1      -0.062747
r_e2      32.000000
r_flux    17.846327
r_size     1.454404
r_x        0.000000
r_y        0.000000
u_e1       0.035465
u_e2      32.000000
u_flux     4.025442
u_size     1.169923
u_x       -0.145644
u_y        0.183524
z_e1      -0.321149
z_e2      32.000000
z_flux    41.126091
z_size     1.576665
z_x        0.047437
z_y       -0.146184
Name: 0, dtype: float64